In [167]:
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from pypfopt import BlackLittermanModel, plotting
from pypfopt import black_litterman, risk_models, objective_functions
from pypfopt.efficient_frontier import EfficientFrontier
from sklearn.ensemble import GradientBoostingRegressor
from pypfopt import risk_models, expected_returns

In [168]:
import warnings

# Disable all warnings
warnings.filterwarnings("ignore")

In [169]:
# Define the list of stock tickers
tickers = ['AAPL', 'AMD', 'AMZN', 'CCJ', 'COST', 'GOOG', 'GS', 'JPM', 'LLY', 'META', 'MSFT', 'NEE', 'PFE', 'SAP', 'WMT']
measurement_days = 10
remaining_days = 30-measurement_days

In [170]:
# Fit non linear ML models to predict stock returns into future

stock_returns = {}
stock_prices = {}
future_returns = {}
short_window_returns = {}

for ticker in tickers:
    data = yf.download(ticker,period="5y")
    print(ticker)
    returns = ((data['Adj Close'] / data['Adj Close'].shift(1))-1).dropna().tolist()
    stock_returns[ticker] = returns

    stock_prices[ticker] = data['Adj Close'].values

    
    prices_x = [returns[i:i+measurement_days] for i in range(len(returns)-remaining_days)]
    prices_y = [returns[i+measurement_days] for i in range(measurement_days,len(returns)-measurement_days)]

    prices_test = [returns[i:i+measurement_days] for i in range(len(returns)-remaining_days,len(returns)-measurement_days)]

    short_window_returns[ticker] = (prices_x, prices_y, prices_test)

    rf = RandomForestRegressor()
    rf.fit(prices_x,prices_y)
    # Predict stock price for 10 future days
    future_returns[ticker] = rf.predict(prices_test)





[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
CCJ
[*********************100%***********************]  1 of 1 completed
COST
[*********************100%***********************]  1 of 1 completed
GOOG
[*********************100%***********************]  1 of 1 completed
GS
[*********************100%***********************]  1 of 1 completed
JPM
[*********************100%***********************]  1 of 1 completed
LLY
[*********************100%***********************]  1 of 1 completed
META
[*********************100%***********************]  1 of 1 completed
MSFT
[*********************100%***********************]  1 of 1 completed
NEE
[*********************100%***********************]  1 of 1 completed
PFE
[*********************100%********************

In [171]:
views = {}
market_caps = {}
for ticker in tickers:
    views[ticker] = future_returns[ticker][-1]
    stock = yf.Ticker(ticker)
    market_caps[ticker] = stock.info["marketCap"]
print(market_caps)

{'AAPL': 3050896097280, 'AMD': 183436115968, 'AMZN': 1337545719808, 'CCJ': 13587570688, 'COST': 238582022144, 'GOOG': 1530754367488, 'GS': 107227783168, 'JPM': 425017868288, 'LLY': 445190078464, 'META': 735452266496, 'MSFT': 2532081664000, 'NEE': 150137765888, 'PFE': 207069970432, 'SAP': 159924043776, 'WMT': 423260553216}


In [172]:
stock_prices = pd.DataFrame(stock_prices)
S = risk_models.CovarianceShrinkage(stock_prices).ledoit_wolf()

In [173]:
market_prices = yf.download("SPY", period="max")["Adj Close"]
delta = black_litterman.market_implied_risk_aversion(market_prices)
print(delta)
market_prior = black_litterman.market_implied_prior_returns(market_caps, delta, S)
print(market_prior)

[*********************100%***********************]  1 of 1 completed
2.595892559847583
AAPL    0.227305
AMD     0.273381
AMZN    0.222945
CCJ     0.140701
COST    0.128900
GOOG    0.209356
GS      0.155310
JPM     0.138731
LLY     0.108788
META    0.245462
MSFT    0.217595
NEE     0.113385
PFE     0.093760
SAP     0.157816
WMT     0.091152
dtype: float64


In [174]:
data = pd.DataFrame()
for ticker in tickers:
    stock = yf.Ticker(ticker)
    df = stock.history(period='5y')  # Retrieve 1-year historical data
    data[ticker] = df['Close']

# Step 2: Calculate historical returns
returns = data.pct_change().dropna()

# Step 3: Calculate statistics
returns_mean = returns.mean()
# Step 4: Assign confidence values based on historical performance
confidence_values = returns_mean.rank() / len(returns_mean)  # Example: Using ranking

# Step 5: Normalize confidence values
confidence_values /= confidence_values.sum()

# Print confidence values
confidence_values = confidence_values.tolist()
print(confidence_values)

[0.10833333333333334, 0.125, 0.03333333333333333, 0.1, 0.08333333333333333, 0.075, 0.041666666666666664, 0.025, 0.11666666666666667, 0.06666666666666667, 0.09166666666666666, 0.058333333333333334, 0.008333333333333333, 0.016666666666666666, 0.05]


In [175]:
bl = BlackLittermanModel(S, pi=market_prior, absolute_views=views, omega="idzorek", view_confidences=confidence_values)
# Get expected returns
ret_bl = bl.bl_returns()
print(ret_bl)

# Get cov matrix
S_bl = bl.bl_cov()

rf_ef = EfficientFrontier(ret_bl, S_bl, weight_bounds=(0.05, 0.1))
rf_ef.add_objective(objective_functions.L2_reg)
rf_ef.min_volatility()
weights = rf_ef.clean_weights()
print(weights)


AAPL    0.153165
AMD     0.170793
AMZN    0.152301
CCJ     0.086769
COST    0.083918
GOOG    0.141316
GS      0.102274
JPM     0.092479
LLY     0.069928
META    0.165189
MSFT    0.145907
NEE     0.074081
PFE     0.063858
SAP     0.105617
WMT     0.061258
dtype: float64
OrderedDict([('AAPL', 0.05621), ('AMD', 0.05), ('AMZN', 0.05718), ('CCJ', 0.06226), ('COST', 0.07761), ('GOOG', 0.05956), ('GS', 0.06391), ('JPM', 0.06845), ('LLY', 0.08038), ('META', 0.05), ('MSFT', 0.05696), ('NEE', 0.07865), ('PFE', 0.0839), ('SAP', 0.06695), ('WMT', 0.08798)])


In [176]:
for weight in weights:
    print(weights[weight] * 800000) # no more than 20% of settled cash

44968.0
40000.0
45744.0
49808.0
62088.0
47648.0
51127.99999999999
54760.0
64303.99999999999
40000.0
45568.0
62920.0
67120.0
53560.0
70384.0


In [177]:
future_returns = {}

for ticker in tickers:
    return_movements = np.array([1 if stock_returns[ticker][i+measurement_days] > stock_returns[ticker][i+measurement_days-1] else 0 for i in range(len(stock_returns[ticker])-measurement_days)])
    return_movements = return_movements[measurement_days:]

    logreg = LogisticRegression()
    logreg.fit(short_window_returns[ticker][0], return_movements)

    future_return_movements = logreg.predict(short_window_returns[ticker][2])
    future_returns[ticker] = [stock_returns[ticker][-1] * (1 + movement) for movement in future_return_movements]

In [178]:
views = {}
for ticker in tickers:
    views[ticker] = future_returns[ticker][-1]


logreg_bl = BlackLittermanModel(S, pi=market_prior, absolute_views=views, omega="idzorek", view_confidences=confidence_values)
# Get expected returns
logreg_ret_bl = logreg_bl.bl_returns()
print(logreg_ret_bl)

# Get cov matrix
logreg_S_bl = logreg_bl.bl_cov()

logreg_ef = EfficientFrontier(logreg_ret_bl, logreg_S_bl, weight_bounds=(0.05, 0.1))
logreg_ef.add_objective(objective_functions.L2_reg)
logreg_ef.min_volatility()
weights = logreg_ef.clean_weights()
print(weights)

AAPL    0.161478
AMD     0.182851
AMZN    0.160396
CCJ     0.094129
COST    0.089714
GOOG    0.148436
GS      0.107803
JPM     0.097410
LLY     0.074239
META    0.173657
MSFT    0.153743
NEE     0.078807
PFE     0.067311
SAP     0.111444
WMT     0.065178
dtype: float64
OrderedDict([('AAPL', 0.05621), ('AMD', 0.05), ('AMZN', 0.05718), ('CCJ', 0.06226), ('COST', 0.07761), ('GOOG', 0.05956), ('GS', 0.06391), ('JPM', 0.06845), ('LLY', 0.08038), ('META', 0.05), ('MSFT', 0.05696), ('NEE', 0.07865), ('PFE', 0.0839), ('SAP', 0.06695), ('WMT', 0.08798)])


In [179]:
future_returns = {}

for ticker in tickers:
    return_movements = np.array([1 if stock_returns[ticker][i+measurement_days] > stock_returns[ticker][i+measurement_days-1] else 0 for i in range(len(stock_returns[ticker])-measurement_days)])
    return_movements = return_movements[measurement_days:]

    svm = SVC(kernel='poly')
    svm.fit(short_window_returns[ticker][0], return_movements)

    future_return_movements = svm.predict(short_window_returns[ticker][2])
    future_returns[ticker] = [stock_returns[ticker][-1] * (1 + movement) for movement in future_return_movements]

In [180]:
views = {}
for ticker in tickers:
    views[ticker] = future_returns[ticker][-1]


svm_bl = BlackLittermanModel(S, pi=market_prior, absolute_views=views, omega="idzorek", view_confidences=confidence_values)
# Get expected returns
svm_ret_bl = svm_bl.bl_returns()
print(svm_ret_bl)

# Get cov matrix
svm_S_bl = svm_bl.bl_cov()

svm_ef = EfficientFrontier(svm_ret_bl, svm_S_bl, weight_bounds=(0.05, 0.1))
svm_ef.add_objective(objective_functions.L2_reg)
svm_ef.min_volatility()
weights = svm_ef.clean_weights()
print(weights)

AAPL    0.161470
AMD     0.182715
AMZN    0.160322
CCJ     0.092760
COST    0.089190
GOOG    0.148307
GS      0.107508
JPM     0.097113
LLY     0.075168
META    0.173612
MSFT    0.154092
NEE     0.078782
PFE     0.067496
SAP     0.111434
WMT     0.065012
dtype: float64
OrderedDict([('AAPL', 0.05621), ('AMD', 0.05), ('AMZN', 0.05718), ('CCJ', 0.06226), ('COST', 0.07761), ('GOOG', 0.05956), ('GS', 0.06391), ('JPM', 0.06845), ('LLY', 0.08038), ('META', 0.05), ('MSFT', 0.05696), ('NEE', 0.07865), ('PFE', 0.0839), ('SAP', 0.06695), ('WMT', 0.08798)])


In [181]:
future_returns = {}

for ticker in tickers:    
    gb = GradientBoostingRegressor()
    gb.fit(short_window_returns[ticker][0],short_window_returns[ticker][1])
    # Predict stock price for 10 future days
    future_returns[ticker] = rf.predict(short_window_returns[ticker][2])

In [182]:
views = {}
for ticker in tickers:
    views[ticker] = future_returns[ticker][-1]


gb_bl = BlackLittermanModel(S, pi=market_prior, absolute_views=views, omega="idzorek", view_confidences=confidence_values)
# Get expected returns
gb_ret_bl = gb_bl.bl_returns()
print(gb_ret_bl)

# Get cov matrix
gb_S_bl = gb_bl.bl_cov()

gb_ef = EfficientFrontier(gb_ret_bl, gb_S_bl, weight_bounds=(0.05, 0.1))
gb_ef.add_objective(objective_functions.L2_reg)
gb_ef.min_volatility()
weights = gb_ef.clean_weights()
print(weights)

AAPL    0.154106
AMD     0.171534
AMZN    0.153352
CCJ     0.087144
COST    0.084724
GOOG    0.142067
GS      0.102752
JPM     0.092876
LLY     0.070395
META    0.166447
MSFT    0.146945
NEE     0.074410
PFE     0.064201
SAP     0.106269
WMT     0.061700
dtype: float64
OrderedDict([('AAPL', 0.05621), ('AMD', 0.05), ('AMZN', 0.05718), ('CCJ', 0.06226), ('COST', 0.07761), ('GOOG', 0.05956), ('GS', 0.06391), ('JPM', 0.06845), ('LLY', 0.08038), ('META', 0.05), ('MSFT', 0.05696), ('NEE', 0.07865), ('PFE', 0.0839), ('SAP', 0.06695), ('WMT', 0.08798)])


In [183]:
rf_ef.portfolio_performance(verbose=True)
print("\n")
logreg_ef.portfolio_performance(verbose=True)
print("\n")
svm_ef.portfolio_performance(verbose=True)
print("\n")
gb_ef.portfolio_performance(verbose=True)



Expected annual return: 10.5%
Annual volatility: 21.8%
Sharpe Ratio: 0.39


Expected annual return: 11.1%
Annual volatility: 21.8%
Sharpe Ratio: 0.42


Expected annual return: 11.1%
Annual volatility: 21.8%
Sharpe Ratio: 0.42


Expected annual return: 10.6%
Annual volatility: 21.8%
Sharpe Ratio: 0.39


(0.10552707889822004, 0.21773691211268306, 0.39280009102893)